# Synthetic Dialogues and Tutoring Simulations

**Project:** Synthetic Data Creation: Survey and Synthesis  
**Method Group:** Educational and Pedagogical Data Synthesis  
**Sub-method:** Synthetic Dialogues and Tutoring Simulations (SDT)  
**Author:** Prajna Penmetsa

**Goal:**  
Generate realistic multi-turn tutoring dialogues between a synthetic teacher and students.  
- Using the Gemini 2.5 Flash REST API, the system simulates conversational exchanges that model how teachers respond to student reasoning, correct misconceptions, and scaffold learning dynamically.

In [1]:
from dotenv import load_dotenv
import os, json, requests, time, re
from tqdm import tqdm

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")
assert API_KEY, "❌ GEMINI_API_KEY not found. Please check your .env file."

MODEL = "gemini-2.5-flash"
URL = f"https://generativelanguage.googleapis.com/v1beta/models/{MODEL}:generateContent?key={API_KEY}"

def call_gemini(prompt):
    payload = {"contents": [{"parts": [{"text": prompt}]}]}
    r = requests.post(URL, json=payload)
    if r.ok:
        return r.json()["candidates"][0]["content"]["parts"][0]["text"]
    else:
        print("❌ Error:", r.status_code, r.text)
        return None

os.makedirs("outputs/sdt", exist_ok=True)

In [2]:
# Load the learners and corresponding feedback data
with open("../synthetic-learner-modeling/outputs/slm/synthetic_learners.json", "r", encoding="utf-8") as f:
    learners = json.load(f)

with open("../feedback-explanation-generation/outputs/feg/feedback_generated.json", "r", encoding="utf-8") as f:
    feedback = json.load(f)

print(f"✅ Loaded {len(learners)} learner profiles and {len(feedback)} feedback records.")

✅ Loaded 5 learner profiles and 5 feedback records.


In [5]:
def make_dialogue_prompt(learner, fb):
    name = learner["student_name"]
    level = learner["learning_level"]
    misconceptions = "; ".join(learner["misconceptions"])
    responses = "\n".join([
        f"Q: {r['question']}\nA: {r['student_answer']}\nReasoning: {r['reasoning']}\nCorrectness: {r['correctness']}"
        for r in learner["responses"]
    ])
    conceptual = fb["conceptual_feedback"]
    example = fb["example_explanation"]
    next_task = fb["next_practice_recommendation"]

    prompt = f"""
You are an educational dialogue simulator.

Simulate a 3–5 turn conversation between a **Teacher** and **Student ({name})**
based on the following context:

Student Level: {level}
Misconceptions: {misconceptions}

Student Responses:
{responses}

Teacher Feedback Summary:
{conceptual}

Example Explanation:
{example}

Next Practice Task:
{next_task}

Dialogue Guidelines:
- Begin with the teacher referencing the student's earlier mistake or reasoning.
- The student should respond naturally — sometimes with confusion, sometimes curiosity.
- The teacher provides clarification, guiding the student to self-correct.
- End the dialogue positively with reinforcement or a next-step suggestion.
- Keep tone supportive, conversational, and educational.

Output in JSON format as:
[
  {{ "speaker": "Teacher", "utterance": "..." }},
  {{ "speaker": "Student", "utterance": "..." }},
  ...
]
"""
    return prompt


In [6]:
dialogues = []

for i, learner in enumerate(tqdm(learners, desc="Generating dialogues")):
    fb = feedback[i]
    prompt = make_dialogue_prompt(learner, fb)
    result = call_gemini(prompt)
    if result:
        dialogues.append({
            "student_name": learner["student_name"],
            "learning_level": learner["learning_level"],
            "dialogue_raw": result
        })
    time.sleep(2)  # pacing

Generating dialogues: 100%|██████████| 5/5 [01:55<00:00, 23.06s/it]


In [7]:
parsed_dialogues = []

for d in dialogues:
    match = re.search(r"\[.*\]", d["dialogue_raw"], re.DOTALL)
    if match:
        try:
            parsed = json.loads(match.group(0))
            parsed_dialogues.append({
                "student_name": d["student_name"],
                "learning_level": d["learning_level"],
                "dialogue": parsed
            })
        except Exception as e:
            print(f"⚠️ Parse error for {d['student_name']}: {e}")

# Save
with open("outputs/sdt/synthetic_tutoring_dialogues.json", "w", encoding="utf-8") as f:
    json.dump(parsed_dialogues, f, indent=2, ensure_ascii=False)

print(f"✅ Parsed and saved {len(parsed_dialogues)} dialogues to outputs/sdt/synthetic_tutoring_dialogues.json")

✅ Parsed and saved 5 dialogues to outputs/sdt/synthetic_tutoring_dialogues.json


In [8]:
for d in parsed_dialogues[:2]:
    print(f"🎓 Dialogue with {d['student_name']} ({d['learning_level']}):\n")
    for turn in d["dialogue"]:
        print(f"{turn['speaker']}: {turn['utterance']}")
    print("-" * 80)

🎓 Dialogue with Alice (beginner):

Teacher: Great job identifying the parts of a fraction for the cake question, Alice! You correctly said that eating 2 slices out of 6 is 2/6. That's a super start. Now, let's look at your first answer. When you compared 1/3 and 1/5, you thought 1/5 was larger because 5 is a bigger number than 3. That's a common thought, but let's use your cake idea to think about it differently. Imagine you have two identical cakes. If you cut one cake into 3 equal slices, and the other into 5 equal slices. Which cake would have bigger individual slices? Think about how much cake you'd get if you took just one slice from each.
Student: Hmm. If I cut a cake into 3 slices, those pieces would be really big! Like, 'wow, that's a lot of cake'. But if I cut it into 5 slices, the pieces would be smaller, right? Because there are more of them. So, a slice from the 3-slice cake would be bigger? But then... 1/3 is bigger than 1/5? Even though 5 is a bigger number?
Teacher: Exac

### Observations & Results

**1. Structural and Conversational Quality**  
- All dialogues were generated in valid JSON format, with 3–5 alternating turns per session.  
- The structure consistently followed a pedagogically sound conversational pattern:  
  - *Teacher identifies misconception → Student reflects → Teacher scaffolds → Student self-corrects → Teacher reinforces next step.*  
- No formatting or semantic inconsistencies were detected across the five dialogues.

**2. Pedagogical Coherence and Adaptivity**  
- **Beginners (Alice, Ben):** Teachers used concrete, relatable analogies (cakes, apples, pizzas) to bridge conceptual gaps.  
- **Intermediates (Carla, David):** Conversations focused on procedural mastery, encouraging verbal reasoning and rule justification.  
- **Advanced (Emily):** The dialogue emphasized metacognition — connecting abstract concepts (scaling effects, real-world meaning) to procedures.  
- This tiered adaptivity showcases how dialogue-based synthesis can emulate real differentiated instruction.

**3. Realism and Engagement**  
- Student utterances captured authentic thought processes — initial uncertainty, progressive reasoning, and “aha” realizations.  
- Teacher responses were conversational, emotionally supportive, and instructive rather than prescriptive, maintaining pedagogical empathy throughout.

**4. Educational Depth**  
- The dialogues collectively demonstrate the three dimensions of effective tutoring:  
  1. **Cognitive Guidance:** Clarifying misconceptions through explanation and example.  
  2. **Affective Support:** Maintaining encouragement and confidence.  
  3. **Actionable Continuity:** Ending with tasks for reinforcement or transfer of learning.  
- This makes the data directly useful for modeling intelligent tutoring systems or dialogue-based learner simulations.

**5. Evaluation Summary**

| Metric | Observation |
|:--|:--|
| Structural fidelity | Excellent – JSON well-formed across all dialogues |
| Conversational flow | Strong – natural teacher–student exchange |
| Pedagogical adaptivity | High – feedback tailored to learner level |
| Cognitive realism | Very High – authentic reasoning and self-correction |
| Emotional tone | Consistent – encouraging and supportive |
| Educational utility | Excellent – ideal for training or evaluating tutoring agents |

**6. Overall Insight**  
The generated tutoring dialogues exhibit high **pedagogical realism** and **instructional coherence**, bridging the gap between feedback and interactive learning.  
This validates synthetic dialogue simulation as a powerful approach for creating *multi-turn, contextually adaptive educational datasets* that mirror authentic teacher–student interactions.

### Run Metadata
- Date: November 4th, 2025  
- Model: `gemini-2.5-flash`  
- Endpoint: `v1beta REST API`  
- Input Source: `synthetic_learners.json` + `feedback_generated.json`  
- Learner Dialogues Simulated: 5  
- Turns per Dialogue: 3–5  
- Output File: `outputs/sdt/synthetic_tutoring_dialogues.json`  
- Temperature: default (~0.9)  
- Author: Prajna Penmetsa  